.. _battery-chargers:

## Battery Chargers

Many modern "smart" battery chargers have data logging capabilities that you can use with Pipecat to view the state of a battery during charging.  Typically, these chargers connect to your computer via a serial port or a serial-over-USB cable that acts like a traditional serial port.  Data is then sent to the port during charging.  For example, data from an iCharger 208B connected to a Mac computer using a USB cable could be read using the following:

In [1]:
# nbconvert: hide
from __future__ import absolute_import, division, print_function

import sys
sys.path.append("../features/steps")

import test
serial = test.mock_module("serial")

serial.serial_for_url.side_effect = test.read_file("../data/icharger208b-charging", stop=3)

In [2]:
import serial
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
for line in port:
    print(line)

$1;1;;12250;3874;93;0;0;0;0;0;0;0;0;291;236;0;20

$1;1;;12250;3875;92;0;0;0;0;0;0;0;0;294;236;0;17

$1;1;;12250;3877;89;0;0;0;0;0;0;0;0;291;236;1;29



Here, we used the pySerial (http://pyserial.readthedocs.io) library to open a serial port and read data from the charger, which sends data to the port one line at a time.  Note that the device name for the port - "/dev/cu.SLAB_USBtoUART" in this case - will vary depending on your device and operating system.  For example, you might use "COM1" or "COM2" on Windows, or "/dev/ttyS0" on Linux.

Our first step in using Pipecat to decipher the raw device output is to turn the for-loop from the previous example into a pipe:

In [3]:
import pipecat.utility
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
for record in pipe:
    print(record)

{'string': u'$1;1;;12250;3874;93;0;0;0;0;0;0;0;0;291;236;0;20\r\n'}
{'string': u'$1;1;;12250;3875;92;0;0;0;0;0;0;0;0;294;236;0;17\r\n'}
{'string': u'$1;1;;12250;3877;89;0;0;0;0;0;0;0;0;291;236;1;29\r\n'}


In this case, :func:`pipecat.utility.readline` converts the raw data into :ref:`records` that store each line of data for further processing.  To decode the contents of each line, we add the appropriate Pipecat device to the end of the pipe:

In [4]:
import pipecat.device.charger
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
for record in pipe:
    print(record)

{('battery', 'cell4', 'voltage'): <Quantity(0.0, 'volt')>, ('charger', 'temperature', 'external'): <Quantity(23.6, 'degC')>, ('battery', 'cell5', 'voltage'): <Quantity(0.0, 'volt')>, ('battery', 'cell8', 'voltage'): <Quantity(0.0, 'volt')>, ('charger', 'mode'): 'charge', ('charger', 'temperature', 'internal'): <Quantity(29.1, 'degC')>, ('battery', 'cell2', 'voltage'): <Quantity(0.0, 'volt')>, ('battery', 'voltage'): <Quantity(3.874, 'volt')>, ('battery', 'cell6', 'voltage'): <Quantity(0.0, 'volt')>, ('battery', 'cell1', 'voltage'): <Quantity(0.0, 'volt')>, ('battery', 'charge'): <Quantity(0.0, 'hour * milliampere')>, ('battery', 'cell3', 'voltage'): <Quantity(0.0, 'volt')>, ('battery', 'cell7', 'voltage'): <Quantity(0.0, 'volt')>, ('battery', 'current'): <Quantity(930.0, 'milliampere')>, ('charger', 'supply'): <Quantity(12.25, 'volt')>}
{('battery', 'cell4', 'voltage'): <Quantity(0.0, 'volt')>, ('charger', 'temperature', 'external'): <Quantity(23.6, 'degC')>, ('battery', 'cell5', 'volt

As you can see, :func:`pipecat.device.charger.icharger208b` has converted the raw data records into charger-specific records containing human-readable fields whose values have appropriate physical units.  Let's use :func:`pipecat.record.dump` to make the output a little more readable:

In [5]:
import pipecat.record
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
for record in pipe:
    pipecat.record.dump(record)

battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 0.0 hour * milliampere
battery/current: 930.0 milliampere
battery/voltage: 3.874 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperature/external: 23.6 degC
charger/temperature/internal: 29.1 degC

battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 0.0 hour * milliampere
battery/current: 920.0 milliampere
battery/voltage: 3.875 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperature/external: 23.6 degC
charger/temperature/internal: 29.4 degC

battery/cell1/voltage: 0.0

Now, you can extract just the data you care about from a record:

In [6]:
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
for record in pipe:
    print(record[("battery", "voltage")], record[("battery", "current")])

3.874 volt 930.0 milliampere
3.875 volt 920.0 milliampere
3.877 volt 890.0 milliampere


And you can convert units safely and explicitly:

In [7]:
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
for record in pipe:
    print(record[("battery", "current")].to(pipecat.units.amps))

0.93 ampere
0.92 ampere
0.89 ampere


As an aside, you may be wondering at this point why it's necessary to explicitly create the serial port and connect it to `readline` ... why not code that functionality directly into `icharger208b`?  The answer is flexibility: by separating Pipecat's functionality into discrete, well-defined components, those components can be easily combined in new and unexpected ways.  For example, you could use `icharger208b` with a charger that communicated over a network socket instead of a serial port.  Or you could "replay" data from a charger stored in a file: 

In [8]:
fobj = open("../data/icharger208b-charging-sample")
pipe = pipecat.utility.readline(fobj)
pipe = pipecat.device.charger.icharger208b(pipe)
for record in pipe:
    print(record[("battery", "charge")])

0.0 hour * milliampere
0.0 hour * milliampere
1.0 hour * milliampere
1.0 hour * milliampere
2.0 hour * milliampere


Let's explore other things we can do with our pipe.  To begin, you might want to add additional metadata to the records returned from a device.  For example, you might want to append timestamps:

In [9]:
# nbconvert: hide
serial.serial_for_url.side_effect = test.read_file("../data/icharger208b-charging", rate=pipecat.quantity(2, pipecat.units.seconds), stop=3)

In [10]:
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
pipe = pipecat.utility.add_timestamp(pipe)
for record in pipe:
    pipecat.record.dump(record)

timestamp: 2017-12-27T00:02:00.421604+00:00
battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 0.0 hour * milliampere
battery/current: 930.0 milliampere
battery/voltage: 3.874 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperature/external: 23.6 degC
charger/temperature/internal: 29.1 degC

timestamp: 2017-12-27T00:02:02.431455+00:00
battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 0.0 hour * milliampere
battery/current: 920.0 milliampere
battery/voltage: 3.875 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperature

Note that :func:`pipecat.utility.add_timestamp` has added a `timestamp` field to each record.  Timestamps in Pipecat are always recorded using UTC (universal) time, so you will likely want to convert them to your local timezone before formatting them for display:

In [11]:
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
pipe = pipecat.utility.add_timestamp(pipe)
for record in pipe:
    print(
        record["timestamp"].to("local").format("YYYY-MM-DD hh:mm:ss a"),
        record[("battery", "voltage")],
    )

2017-12-26 05:02:06 pm 3.874 volt
2017-12-26 05:02:08 pm 3.875 volt
2017-12-26 05:02:10 pm 3.877 volt


You could also use :func:`pipecat.utility.add_field` to append your own custom field to every record that passes through the pipe.

Now let's consider calculating some simple statistics, such as the minimum and maximum battery voltage while charging.  When we iterate over the contents of a pipe using a `for` loop, we receive one record at-a-time until the pipe is empty.  We could keep track of a "running" minimum and maximum during iteration, and there are use-cases where that is the best way to solve the problem.  However, for moderately-sized data, Pipecat provides a more convenient approach: 

In [12]:
# nbconvert: hide
serial.serial_for_url.side_effect = test.read_file("../data/icharger208b-charging")

In [13]:
import pipecat.store
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
pipe = pipecat.store.cache(pipe)
for record in pipe:
    pass
print(len(pipe.table))
print(pipe.table[("battery", "voltage")])

1390
[ 3.874  3.875  3.877 ...,  4.119  4.119  4.119] volt


Here, :func:`pipecat.store.cache` creates an in-memory cache that stores every record it receives.   We have a do-nothing `for` loop that reads data from the charger to populate the cache.  Once that's complete, we can use the cache `table` attribute to retrieve data from the cache using the same keys and syntax we would use with a record.  Unlike a record, the cache returns every value for a given key at once (using a Numpy array), which makes it easy to compute the statistics we're interested in:

In [14]:
import pipecat.store
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
pipe = pipecat.store.cache(pipe)
for record in pipe:
    pass
print("Min:", pipe.table[("battery", "voltage")].min())
print("Max:", pipe.table[("battery", "voltage")].max())

Min: 3.874 volt
Max: 4.152 volt


Consolidating fields using the cache is also perfect for generating plots with a library like Toyplot (http://toyplot.readthedocs.io):

In [15]:
import toyplot

canvas = toyplot.Canvas()
axes = canvas.cartesian(grid=(3, 1, 0), label="Battery", ylabel="Voltage (V)")
axes.plot(pipe.table[("battery", "voltage")].to(pipecat.units.volt))

axes = canvas.cartesian(grid=(3, 1, 1), ylabel="Current (A)")
axes.plot(pipe.table[("battery", "current")].to(pipecat.units.amp))

axes = canvas.cartesian(grid=(3, 1, 2), ylabel="Charge (mAH)")
axes.plot(pipe.table[("battery", "charge")].to(pipecat.units.milliamp * pipecat.units.hour));

<path d="M 50.0 142.05298013245033 L 50.333333333333329 141.7218543046358 L 50.666666666666671 141.05960264900673 L 51.0 141.05960264900673 L 51.333333333333336 141.05960264900673 L 51.666666666666671 141.05960264900673 L 52.0 140.72847682119206 L 52.333333333333336 140.3973509933775 L 52.666666666666664 140.3973509933775 L 53.0 140.3973509933775 L 53.333333333333329 139.73509933774844 L 53.666666666666664 139.73509933774844 L 54.0 139.73509933774844 L 54.333333333333329 138.07947019867549 L 54.666666666666664 137.41721854304643 L 55.0 138.74172185430473 L 55.333333333333329 138.74172185430473 L 55.666666666666671 137.41721854304643 L 56.0 137.41721854304643 L 56.333333333333336 137.41721854304643 L 56.666666666666671 138.07947019867549 L 57.0 137.41721854304643 L 57.333333333333329 138.07947019867549 L 57.666666666666671 137.41721854304643 L 58.0 138.07947019867549 L 58.333333333333329 134.43708609271533 L 58.666666666666664 137.41721854304643 L 59.0 137.41721854304643 L 59.333333333333329 137.41721854304643 L 59.666666666666664 137.41721854304643 L 60.0 137.41721854304643 L 60.333333333333336 137.41721854304643 L 60.666666666666671 137.41721854304643 L 61.0 137.0860927152319 L 61.333333333333336 136.42384105960267 L 61.666666666666671 137.0860927152319 L 62.0 136.42384105960267 L 62.333333333333329 136.42384105960267 L 62.666666666666671 136.42384105960267 L 62.999999999999993 136.42384105960267 L 63.333333333333343 136.42384105960267 L 63.666666666666664 136.42384105960267 L 64.0 136.42384105960267 L 64.333333333333343 134.76821192052986 L 64.666666666666657 135.0993377483444 L 65.0 135.7615894039736 L 65.333333333333343 135.7615894039736 L 65.666666666666671 135.7615894039736 L 66.0 135.7615894039736 L 66.333333333333329 135.0993377483444 L 66.666666666666671 134.43708609271533 L 67.0 133.77483443708613 L 67.333333333333329 134.43708609271533 L 67.666666666666671 133.77483443708613 L 68.0 134.43708609271533 L 68.333333333333343 134.43708609271533 L 68.666666666666671 134.43708609271533 L 69.0 134.43708609271533 L 69.333333333333343 134.43708609271533 L 69.666666666666657 134.43708609271533 L 70.0 134.43708609271533 L 70.333333333333329 134.43708609271533 L 70.666666666666657 133.77483443708613 L 71.0 134.43708609271533 L 71.333333333333329 133.77483443708613 L 71.666666666666671 133.77483443708613 L 72.0 130.79470198675503 L 72.333333333333329 134.43708609271533 L 72.666666666666671 133.77483443708613 L 73.0 133.77483443708613 L 73.333333333333343 133.77483443708613 L 73.666666666666657 130.13245033112582 L 74.0 130.79470198675503 L 74.333333333333343 133.77483443708613 L 74.666666666666657 133.77483443708613 L 75.0 133.77483443708613 L 75.333333333333343 133.77483443708613 L 75.666666666666657 133.77483443708613 L 76.0 133.77483443708613 L 76.333333333333343 133.4437086092716 L 76.666666666666671 133.77483443708613 L 77.0 133.77483443708613 L 77.333333333333329 133.77483443708613 L 77.666666666666671 133.77483443708613 L 78.0 133.4437086092716 L 78.333333333333343 133.4437086092716 L 78.666666666666671 133.4437086092716 L 79.0 133.4437086092716 L 79.333333333333343 132.78145695364236 L 79.666666666666657 132.78145695364236 L 80.0 132.78145695364236 L 80.333333333333343 133.4437086092716 L 80.666666666666657 131.12582781456956 L 81.0 132.45033112582783 L 81.333333333333329 132.45033112582783 L 81.666666666666671 132.45033112582783 L 82.0 132.45033112582783 L 82.333333333333329 132.1192052980133 L 82.666666666666657 132.1192052980133 L 83.0 130.13245033112582 L 83.333333333333329 132.1192052980133 L 83.666666666666657 132.1192052980133 L 84.0 132.1192052980133 L 84.333333333333343 132.1192052980133 L 84.666666666666657 132.1192052980133 L 85.0 130.79470198675503 L 85.333333333333343 131.45695364238409 L 85.666666666666657 132.1192052980133 L 86.0 130.79470198675503 L 86.333333333333343 131.45695364238409 L 86.666666666666657 132.1192052980133 L 87.0 131.45695364238409 L 87.333333333333343 131.45695364238409 L 87.66666666666

Note that nothing prevents us from doing useful work in the `for` loop that populates the cache, and nothing prevents us from accessing the cache within the loop.  For example, we might want to display field values from individual records alongside a running average computed from the cache.  Or we might want to update our plot periodically as the loop progresses.

Moving on, you will likely want to store records to disk for later access.  Pipecat provides components to make this easy too.  First, you can add :func:`pipecat.store.pickle.write` to the end of a pipe, to write records to disk using Python's pickle format:

In [16]:
# nbconvert: hide
import os

serial.serial_for_url.side_effect = test.read_file("../data/icharger208b-charging", rate=pipecat.quantity(2, pipecat.units.seconds), stop=3)
if os.path.exists("charger.pickle"):
    os.remove("charger.pickle")

In [17]:
import pipecat.store.pickle
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
pipe = pipecat.utility.add_timestamp(pipe)
pipe = pipecat.store.pickle.write(pipe, "charger.pickle")
for record in pipe:
    pass

In [18]:
pipe = pipecat.store.pickle.read("charger.pickle")
pipe = pipecat.store.cache(pipe)
for record in pipe:
    pipecat.record.dump(record)
print("Average:", pipe.table[("battery", "voltage")].mean())

timestamp: 2017-12-27T00:02:14.993602+00:00
battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 0.0 hour * milliampere
battery/current: 930.0 milliampere
battery/voltage: 3.874 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperature/external: 23.6 degC
charger/temperature/internal: 29.1 degC

timestamp: 2017-12-27T00:02:17.001294+00:00
battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 0.0 hour * milliampere
battery/current: 920.0 milliampere
battery/voltage: 3.875 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperature

This is another example of the interchangeability of the Pipecat components: the pickle writer is a record consumer, and the pickle reader is a record generator.  In essence, we "broke" our previous pipe into two separate pipes that communicate via the filesystem.  While we won't go into detail here, a similar approach could be used to communicate between threads using a message queue or between processes over a socket.

In [19]:
# nbconvert: hide
serial.serial_for_url.side_effect = test.read_file("../data/icharger208b-charging", rate=pipecat.quantity(2, pipecat.units.seconds), stop=10, block=True)

There is one final issue that we've ignored so far: when to stop logging data.  The :func:`pipecat.utility.readline()` function will read data from the serial port as long as the port is open, blocking indefinitely if no data is arriving.  That means that for all the preceeding examples the `for` loop will never end unless the serial port is closed (i.e. the external device is turned-off or unplugged), or the code is interrupted using Control-C.  While that's fine for prototyping at the command line, we need to have a way to stop collecting data and shutdown cleanly if we're going to automate data logging processes.  Fortunately, Pipecat provides several easy-to-use functions to do just that:

In [20]:
import pipecat.limit
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
pipe = pipecat.utility.add_timestamp(pipe)
pipe = pipecat.limit.count(pipe, count=2)
for record in pipe:
    pipecat.record.dump(record)

timestamp: 2017-12-27T00:02:21.056456+00:00
battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 0.0 hour * milliampere
battery/current: 930.0 milliampere
battery/voltage: 3.874 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperature/external: 23.6 degC
charger/temperature/internal: 29.1 degC

timestamp: 2017-12-27T00:02:23.066381+00:00
battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 0.0 hour * milliampere
battery/current: 920.0 milliampere
battery/voltage: 3.875 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperature

Here, :func:`pipecat.limit.count` ends the loop when `count` records have been received.  This is often handy during development to limit the amount of data consumed from a device that produces output continuously.  However, this approach is no good for devices like our charger that will produce a finite, indeterminate number of records - if the device stops sending records before the `count` has been reached, the loop will still block.  Instead, we could use :func:`pipecat.limit.duration` to limit the total amount of time the loop is allowed to run instead:

In [21]:
import pipecat.limit
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
pipe = pipecat.utility.add_timestamp(pipe)
pipe = pipecat.limit.duration(pipe, duration=pipecat.quantity(4, pipecat.units.seconds))
for record in pipe:
    pipecat.record.dump(record)

timestamp: 2017-12-27T00:02:23.079796+00:00
battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 0.0 hour * milliampere
battery/current: 930.0 milliampere
battery/voltage: 3.874 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperature/external: 23.6 degC
charger/temperature/internal: 29.1 degC

timestamp: 2017-12-27T00:02:25.085261+00:00
battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 0.0 hour * milliampere
battery/current: 920.0 milliampere
battery/voltage: 3.875 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperature

This approach is an improvement because it puts an upper-bound on the amount of time the loop will run, whether the device has stopped sending records or not.  However, it's still error-prone, since we don't know in advance how long charging will take - if we set the duration too low, it may stop the loop before charging is complete.  If we set the duration too high, we will capture all the records we want, but we will likely waste time waiting for records that will never come.  Ideally, we would like to exit the loop as soon as the charger tells us it's finished.  Fortunately, the charger provides a field - `charger/mode` that can do just that:

In [22]:
# nbconvert: hide
serial.serial_for_url.side_effect = test.read_file("../data/icharger208b-charging", rate=pipecat.quantity(2, pipecat.units.seconds), start=1285, block=True)

In [23]:
import pipecat.limit
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
pipe = pipecat.utility.add_timestamp(pipe)
pipe = pipecat.limit.until(pipe, key=("charger", "mode"), value="finished")
for record in pipe:
    pipecat.record.dump(record)

timestamp: 2017-12-27T00:02:27.114364+00:00
battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 503.0 hour * milliampere
battery/current: 110.0 milliampere
battery/voltage: 4.141 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperature/external: 23.8 degC
charger/temperature/internal: 32.5 degC

timestamp: 2017-12-27T00:02:29.121514+00:00
battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 503.0 hour * milliampere
battery/current: 120.0 milliampere
battery/voltage: 4.14 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperat

:func:`pipecat.limit.until` terminates the loop as soon as it receives a record with the given key and value.  This approach finally gets us our desired behavior (loop ends as soon as the charger is finished), but it could use just a little more work to make it robust.  For example, what happens if the charger stops sending data before the mode changes?  We could combine :func:`pipecat.limit.until` with :func:`pipecat.limit.duration`, but that would still suffer from the terminate-too-soon / waste-too-much-time problem.  Fortunately, we know from testing that our charger sends records every two seconds (if at all), and Pipecat provides :func:`pipecat.limit.timeout`, which can terminate the loop if it doesn't receive a record within a specified time interval: 

In [24]:
import pipecat.limit
port = serial.serial_for_url("/dev/cu.SLAB_USBtoUART", baudrate=128000)
pipe = pipecat.utility.readline(port)
pipe = pipecat.device.charger.icharger208b(pipe)
pipe = pipecat.utility.add_timestamp(pipe)
pipe = pipecat.limit.until(pipe, key=("charger", "mode"), value="finished")
pipe = pipecat.limit.timeout(pipe, timeout=pipecat.quantity(5, pipecat.units.seconds))
for record in pipe:
    pipecat.record.dump(record)

timestamp: 2017-12-27T00:02:31.141096+00:00
battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 503.0 hour * milliampere
battery/current: 110.0 milliampere
battery/voltage: 4.141 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperature/external: 23.8 degC
charger/temperature/internal: 32.5 degC

timestamp: 2017-12-27T00:02:33.145768+00:00
battery/cell1/voltage: 0.0 volt
battery/cell2/voltage: 0.0 volt
battery/cell3/voltage: 0.0 volt
battery/cell4/voltage: 0.0 volt
battery/cell5/voltage: 0.0 volt
battery/cell6/voltage: 0.0 volt
battery/cell7/voltage: 0.0 volt
battery/cell8/voltage: 0.0 volt
battery/charge: 503.0 hour * milliampere
battery/current: 120.0 milliampere
battery/voltage: 4.14 volt
charger/mode: charge
charger/supply: 12.25 volt
charger/temperat

This example still exits normally when the charger is finished, but it will also exit cleanly if a record isn't received within a five seconds.  More importantly, you can see how easy it is to combine limit functions to control when the `for` loop terminates.